In [1]:
import numpy as np
import pandas as pd

In [2]:
train_data=[["Sunny","Hot","High","Weak",0],["Sunny","Hot","High","Strong",0],\
      ["Overcast","Hot","High","Weak",1],["Rain","Mild","High","Weak",1],\
      ["Rain","Cool","Normal","Weak",1],["Rain","Cool","Normal","Strong",0],\
      ["Overcast","Cool","Normal","Strong",1],["Sunny","Mild","High","Weak",0],\
      ["Sunny","Cool","Normal","Weak",1],["Rain","Mild","Normal","Weak",1],\
      ["Sunny","Mild","Normal","Strong",1],["Overcast","Mild","High","Strong",1],\
      ["Overcast","Hot","Normal","Weak",1],["Rain","Mild","High","Strong",0]]
data=pd.DataFrame(train_data,columns=['Outlook',"Temperature","Humidity","Wind","PlayTennis"])
print(data)

     Outlook Temperature Humidity    Wind  PlayTennis
0      Sunny         Hot     High    Weak           0
1      Sunny         Hot     High  Strong           0
2   Overcast         Hot     High    Weak           1
3       Rain        Mild     High    Weak           1
4       Rain        Cool   Normal    Weak           1
5       Rain        Cool   Normal  Strong           0
6   Overcast        Cool   Normal  Strong           1
7      Sunny        Mild     High    Weak           0
8      Sunny        Cool   Normal    Weak           1
9       Rain        Mild   Normal    Weak           1
10     Sunny        Mild   Normal  Strong           1
11  Overcast        Mild     High  Strong           1
12  Overcast         Hot   Normal    Weak           1
13      Rain        Mild     High  Strong           0


In [3]:
cols=data.shape[1]
x_data=data.iloc[:,:cols-1]
y_data=data.iloc[:,cols-1:]
featurenames=x_data.columns

In [4]:
def Navie_Bayes(X_data,Y_data):
    y=Y_data.values
    x=X_data.values
    y_unique=np.unique(y)
    prior_prob=np.zeros(len(y_unique))
    for i in range(len(y_unique)):
        prior_prob[i]=sum(y==y_unique[i])/len(y)

    condition_prob={}
    for feat in featurenames:
        x_unique=list(set(X_data[feat]))
        
        x_condition_prob=np.zeros((len(y_unique),len(x_unique)))
        for j in range(len(y_unique)):
            for k in range(len(x_unique)):
                x_condition_prob[j,k]=sum((X_data[feat]==x_unique[k])&(Y_data.PlayTennis==y_unique[j]))/sum(y==y_unique[j])
        x_condition_prob=pd.DataFrame(x_condition_prob,columns=x_unique,index=y_unique)
        condition_prob[feat]=x_condition_prob
    return prior_prob,condition_prob

In [5]:
def Prediction(testData,prior,condition_prob):
    numclass=prior.shape[0]
    numsample=testData.shape[0]
    featureNames=testData.columns
    post_prob=np.zeros((numsample,numclass))
    for k in range(numsample):
        prob_k=np.zeros((numclass,))
        for i in range(numclass):
            pri=prior[i]
            for feat in featureNames:
                feat_val=testData[feat][k]
                cp=condition_prob[feat]
                cp_val=cp.loc[i,feat_val]
                pri*=cp_val
            prob_k[i]=pri
        prob=prob_k/np.sum(prob_k,axis=0)
        post_prob[k,:]=prob
    return post_prob

In [6]:
x_test=pd.DataFrame([["Sunny","Cool","High","Strong"]],columns=['Outlook',"Temperature","Humidity","Wind"])
print(x_test)

  Outlook Temperature Humidity    Wind
0   Sunny        Cool     High  Strong


In [7]:
prior_prob,condition_prob=Navie_Bayes(x_data,y_data)
print(prior_prob)
print(condition_prob)

[0.35714286 0.64285714]
{'Outlook':       Sunny  Overcast      Rain
0  0.600000  0.000000  0.400000
1  0.222222  0.444444  0.333333, 'Temperature':        Cool      Mild       Hot
0  0.200000  0.400000  0.400000
1  0.333333  0.444444  0.222222, 'Humidity':        High    Normal
0  0.800000  0.200000
1  0.333333  0.666667, 'Wind':      Strong      Weak
0  0.600000  0.400000
1  0.333333  0.666667}


C:\Users\Gillian\AppData\Local\Temp\ipykernel_4692\3747978167.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prior_prob[i]=sum(y==y_unique[i])/len(y)
C:\Users\Gillian\AppData\Local\Temp\ipykernel_4692\3747978167.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  x_condition_prob[j,k]=sum((X_data[feat]==x_unique[k])&(Y_data.PlayTennis==y_unique[j]))/sum(y==y_unique[j])


In [8]:
postProir=Prediction(x_test,prior_prob,condition_prob)
print(postProir)

[[0.79541735 0.20458265]]


In [9]:
if(postProir[0][0]>postProir[0][1]):
    print("不可打球")
else:
    print("可打球")

不可打球
